In [1]:
import numpy as np

In [25]:
from sklearn.datasets import make_spd_matrix
N = 1000
D = 2
mu = np.random.uniform(-1, 1, D)
Sigma = make_spd_matrix(D)

from scipy.stats import multivariate_normal
X = multivariate_normal.rvs(mu, Sigma, N)

from random import randint
P = 1; Q = 1
t_o = randint(max(P, Q), N)

Beta = np.random.uniform(-1, 1, D)
Phi = np.random.uniform(-1, 1, P)
Theta = np.random.uniform(-1, 1, Q)

In [26]:
alpha = 15
t1 = []; t2 = []
nu = list(np.random.uniform(0,2, max(P, Q)))
Y = []
for i in range(max(P, Q)):
    x = np.random.poisson(np.exp(nu[i]))
    while x==0:
        x = np.random.poisson(np.exp(nu[i]))
    Y.append(x)
    t1.append(np.log(Y[i]) - np.dot(X[i], Beta))
    t2.append(np.log(Y[i]) - nu[i])
 
for i in range(max(P, Q), N):
    nu_ = np.dot(X[i], Beta) + np.dot(Phi, t1[:(-P-1):-1]) + np.dot(Theta, t2[:(-Q-1):-1])
    #print(np.exp(nu_))
    nu.append(nu_)
    try:
        if i!=(t_o - 1):
            x = np.random.poisson(np.exp(nu_))
            while x==0:
                x = np.random.poisson(np.exp(nu_))
            Y.append(x)
        else:
            x = np.random.poisson(alpha*np.exp(nu_))
            while x==0:
                x = np.random.poisson(alpha*np.exp(nu[i]))
            Y.append(x)

        if i<(N-1):
            t1.append(np.log(x) - np.dot(X[i+1], Beta))
            t2.append(np.log(x) - nu_)
    except:
        break

In [27]:
Y = np.array(Y)

In [28]:
def cal_mu(Beta, Phi, Theta, X, Y, mu):
    """
    X -> Observed X matrix NxD
    Y -> Observed Y
    mu -> Initial values of mu
    """
    P = len(Phi); Q = len(Theta)
    N = X.shape[0]
    mu = list(mu)
    t1 = []; t2 = []
    for i in range(max(P, Q)):
        t1.append(np.log(Y[i]) - np.dot(X[i], Beta))
        t2.append(np.log(Y[i]) - mu[i])
    for i in range(max(P, Q), N):
        mu_ = np.dot(X[i], Beta) + np.dot(Phi, t1[:(-P-1):-1]) + np.dot(Theta, t2[:(-Q-1):-1])
        mu.append(np.exp(mu_))
        if i<(N-1):
            t1.append(np.log(Y[i]) - np.dot(X[i+1], Beta))
            t2.append(np.log(Y[i]) - mu_)
    mu = np.array(mu)
    return mu

In [10]:
import pickle

In [14]:
with open('../Downloads/params.pkl','rb') as f:
    params = pickle.load(f)

In [32]:

# coding: utf-8

# In[1]:


import numpy as np
import scipy as sp
#import statsmodels.api as sm
from sklearn.linear_model import LinearRegression


c=2
h=0.5

def loggarma_null(X,Y,p,q,max_iter,t_0):
    z = np.array([c]*Y.shape[0])
    Y_1 = np.maximum(Y,z)
    beta = [0]*(X.shape[1] - 1)
    beta.insert(0,np.log(Y.sum()/Y.shape[0]))
    beta = np.array(beta)
    phi = np.random.rand(p).reshape(-1,1)
    theta = np.random.rand(q).reshape(-1,1)
    alpha = 1
    eta = np.random.rand(Y.shape[0],1)
    deta_beta = np.random.rand(Y.shape[0],beta.shape[0])
    deta_phi = np.random.rand(Y.shape[0],p)
    deta_theta = np.random.rand(Y.shape[0],q)
    deta_alpha = np.zeros([Y.shape[0],1])
    deta_alpha[t_0]= 0
    max_num = max(max(p,q),beta.shape[0])

    for i in range(max_iter):
        #Save old values 
        dold_beta = deta_beta
        dold_phi = deta_phi
        dold_theta = deta_theta
        dold_alpha = deta_alpha
        eta_old = eta
      
        #calculate new values of eta
        eta[:max_num] = np.log(Y[:max_num].reshape(-1,1))
        eta[max_num:] = np.dot(X[max_num:,:],beta).reshape(-1,1)
        
        eta[t_0] += np.log(1)
        for j in range(X.shape[0] - max_num):
            X_block_p = X[j:j + phi.shape[0],:].transpose().copy()
            Y_block_q = Y[j:j + theta.shape[0]].copy()
            Y_block_p = Y[j:j + phi.shape[0]].copy()
            eta_block_q = eta_old[j:j+theta.shape[0]].copy()
                        
            phi_block = np.log(np.flip(Y_block_p,axis=0).reshape(-1,1)) - np.dot(np.flip(X_block_p,axis=0).T,beta).reshape(-1,1)
            theta_block = np.log(np.flip(Y_block_q,axis=0).reshape(-1,1)) - eta_block_q
            eta[j + max_num] += (np.inner(phi_block.T,phi.T) + np.inner(theta_block.T,theta.T)).reshape(-1,)
            
##Update gradients
        deta_beta = X[:,0:beta.shape[0]].copy()
        for j in range(X.shape[0] - phi.shape[0]):
            X_block = X[j:j + phi.shape[0],:].copy()

            Y_block = Y[j:j+ phi.shape[0]].copy()
            
            deta_beta[j+phi.shape[0],:] -= (np.inner(np.flip(phi,axis=0).T, X_block.T).T).reshape(-1,)

            deta_phi[j+phi.shape[0],:] = np.log(np.flip(Y_block,axis=0)) - (np.dot(np.flip(X_block,axis=0),beta)).reshape(-1)
    
        for j in range(X.shape[0] - theta.shape[0]):
            # q-sized blocks of older gradients11
            dbeta_block = dold_beta[j:j+theta.shape[0],:].transpose().copy()
            dphi_block = dold_phi[j:j+theta.shape[0],:].transpose().copy()
            dtheta_block = dold_theta[j:j+theta.shape[0],:].transpose().copy()

            # update after multiplying with current values of theta.
            Y_block_q = Y[j:j + theta.shape[0]].copy()
            eta_block_q = eta[j:j + theta.shape[0]].copy()
            deta_theta[j+theta.shape[0],:] = np.log(Y_block_q) - np.flip(eta_block_q.T,axis=0)
            deta_beta[j+theta.shape[0],:] -= (np.inner(np.flip(theta,axis=0).transpose(),dbeta_block).transpose()).reshape(-1)
            deta_phi[j+theta.shape[0],:] -= (np.inner(np.flip(theta,axis=0).transpose(),dphi_block).transpose()).reshape(-1)
            deta_theta[j+theta.shape[0],:] -= (np.inner(np.flip(theta,axis=0).transpose(),dtheta_block).transpose()).reshape(-1)
        
        deta_alpha = np.zeros([Y.shape[0],1])
        deta_alpha[t_0]= 0
            
   
#OLS minimization
        mu = np.exp(eta)
        mu = np.clip(mu, 0.1,10e30)
        
        
        R = np.dot(deta_beta,beta).reshape(-1,1) + np.dot(deta_phi,phi)+ np.dot(deta_theta,theta)  + h*(Y.reshape(-1,1) - mu)*mu
        X_R = np.concatenate((deta_beta,deta_phi,deta_theta),axis = 1)
        
        R = np.clip(R, -10e10,10e10)
        X_R = np.clip(X_R, -10e10,10e10)
#        wls = sm.WLS(R,X_R,weights = mu, missing='drop')
#        res_wls = wls.fit('qr')
        model = LinearRegression()
        model.fit(X_R,R,mu.reshape(-1))
        params = (model.coef_).reshape(X_R.shape[1],1)
        if (i+1)%10 == 0:
            print('iteration: ', i)
            print(model.coef_)
            
#        if np.sum(np.isnan(res_wls.params)) == 0:
#            print(res_wls.params)
            
            beta = params[:beta.shape[0]]
            phi = params[beta.shape[0]:p+beta.shape[0]]
            theta = params[beta.shape[0]+p:beta.shape[0]+p+q]
            print(params.shape)
            alpha = float(1)
            
            saveDict = {'beta':beta, 'phi':phi, 'theta':theta, 'alpha':alpha}
            print(saveDict)
#             import pickle
#             with open('params.pkl', 'wb') as f:
#                 pickle.dump(f,saveDict)
        
    return beta,phi,theta,alpha
beta0, phi0, theta0, alpha0 = loggarma_null(X=X,Y=Y,p=2,q=4,max_iter=50,t_0=50)


# In[8]:


beta0, phi0, theta0, alpha0 = loggarma_null(X=X,Y=Y,p=2,q=4,max_iter=50,t_0=50)

iteration:  9
[[-5.42683470e+05  8.21458109e+04 -2.03394509e+05 -6.14237040e+04
   1.79563280e-01 -3.12096812e-02  2.35604494e-02  1.12837070e-02]]
(8, 1)
{'beta': array([[-542683.47018161],
       [  82145.81089688]]), 'phi': array([[-203394.50866564],
       [ -61423.70398896]]), 'theta': array([[ 0.17956328],
       [-0.03120968],
       [ 0.02356045],
       [ 0.01128371]]), 'alpha': 1.0}
iteration:  19
[[ 5.22606140e-16 -7.91066999e-17 -1.95869571e-16 -5.91514372e-17
  -1.22556744e-27 -2.30327913e-27 -1.67179118e-27 -9.36179308e-28]]
(8, 1)
{'beta': array([[ 5.22606140e-16],
       [-7.91066999e-17]]), 'phi': array([[-1.95869571e-16],
       [-5.91514372e-17]]), 'theta': array([[-1.22556744e-27],
       [-2.30327913e-27],
       [-1.67179118e-27],
       [-9.36179308e-28]]), 'alpha': 1.0}
iteration:  29
[[-0.06011756  0.32995062  0.9839169   0.06868376 -0.14879971  0.63085093
  -1.11011169  1.34512803]]
(8, 1)
{'beta': array([[-0.06011756],
       [ 0.32995062]]), 'phi': array([[0

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: RuntimeWarning: invalid value encountered in subtract


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [19]:
mu_o = np.exp(np.array(nu[:max(P, Q)]))
mu_o = cal_mu(Beta, Phi, Theta, X, Y, mu_o)

In [21]:
def pearson_residual(Y, mu):
    return (Y-mu)/(mu**0.5)
N = 100
k = int(N*0.05)
pear_res = pearson_residual(Y, mu_o)
A = np.absolute(pear_res).argsort()[:-k:-1]

In [ ]:
c=2
h=0.5

def loggarma(X,Y,p,q,max_iter,t_0):
    z = np.array([c]*Y.shape[0])
    Y_1 = np.maximum(Y,z)
    beta = [0]*(X.shape[1] - 1)
    beta.insert(0,np.log(Y.sum()/Y.shape[0]))
    beta = np.array(beta)
    phi = np.random.rand(p).reshape(-1,1)
    theta = np.random.rand(q).reshape(-1,1)
    alpha = 1
    eta = np.random.rand(Y.shape[0],1)
    deta_beta = np.random.rand(Y.shape[0],beta.shape[0])
    deta_phi = np.random.rand(Y.shape[0],p)
    deta_theta = np.random.rand(Y.shape[0],q)
    deta_alpha = np.random.rand(Y.shape[0],1)
    deta_alpha[t_0]= 1./alpha
    max_num = max(max(p,q),beta.shape[0])

    for i in range(max_iter):
    #Save old values 
    dold_beta = deta_beta
    dold_phi = deta_phi
    dold_theta = deta_theta
    dold_alpha = deta_alpha
    eta_old = eta

    #calculate new values of eta
    eta[:max_num] = np.log(Y[:max_num].reshape(-1,1))
    eta[max_num:] = np.dot(X[max_num:,:],beta).reshape(-1,1)

    eta[t_0] += np.log(alpha)
    for j in range(X.shape[0] - max_num):
    X_block_p = X[j:j + phi.shape[0],:].transpose().copy()
    Y_block_q = Y[j:j + theta.shape[0]].copy()
    Y_block_p = Y[j:j + phi.shape[0]].copy()
    eta_block_q = eta_old[j:j+theta.shape[0]].copy()

    phi_block = np.log(np.flip(Y_block_p,axis=0).reshape(-1,1)) - np.dot(np.flip(X_block_p,axis=0).T,beta).reshape(-1,1)
    theta_block = np.log(np.flip(Y_block_q,axis=0).reshape(-1,1)) - eta_block_q
    eta[j + max_num] += (np.inner(phi_block.T,phi.T) + np.inner(theta_block.T,theta.T)).reshape(-1,)

    ##Update gradients
    deta_beta = X[:,0:beta.shape[0]].copy()
    for j in range(X.shape[0] - phi.shape[0]):
    X_block = X[j:j + phi.shape[0],:].copy()

    Y_block = Y[j:j+ phi.shape[0]].copy()

    deta_beta[j+phi.shape[0],:] -= (np.inner(np.flip(phi,axis=0).T, X_block.T).T).reshape(-1,)

    deta_phi[j+phi.shape[0],:] = np.log(np.flip(Y_block,axis=0)) - (np.dot(np.flip(X_block,axis=0),beta)).reshape(-1)

    for j in range(X.shape[0] - theta.shape[0]):
    # q-sized blocks of older gradients11
    dbeta_block = dold_beta[j:j+theta.shape[0],:].transpose().copy()
    dphi_block = dold_phi[j:j+theta.shape[0],:].transpose().copy()
    dtheta_block = dold_theta[j:j+theta.shape[0],:].transpose().copy()

    # update after multiplying with current values of theta.
    Y_block_q = Y[j:j + theta.shape[0]].copy()
    eta_block_q = eta[j:j + theta.shape[0]].copy()
    deta_theta[j+theta.shape[0],:] = np.log(Y_block_q) - np.flip(eta_block_q.T,axis=0)
    deta_beta[j+theta.shape[0],:] -= (np.inner(np.flip(theta,axis=0).transpose(),dbeta_block).transpose()).reshape(-1)
    deta_phi[j+theta.shape[0],:] -= (np.inner(np.flip(theta,axis=0).transpose(),dphi_block).transpose()).reshape(-1)
    deta_theta[j+theta.shape[0],:] -= (np.inner(np.flip(theta,axis=0).transpose(),dtheta_block).transpose()).reshape(-1)

    deta_alpha = np.zeros([Y.shape[0],1])
    deta_alpha[t_0]= 1./alpha


    #OLS minimization
    mu = np.exp(eta)
    mu = np.clip(mu, 0.1,10e30)


    R = np.dot(deta_beta,beta).reshape(-1,1) + np.dot(deta_phi,phi)+ np.dot(deta_theta,theta) + np.dot(deta_alpha,alpha) + h*(Y.reshape(-1,1) - mu)*mu
    X_R = np.concatenate((deta_beta,deta_phi,deta_theta,deta_alpha),axis = 1)

    R = np.clip(R, -10e10,10e10)
    X_R = np.clip(X_R, -10e10,10e10)
    # wls = sm.WLS(R,X_R,weights = mu, missing='drop')
    # res_wls = wls.fit('qr')
    model = LinearRegression()
    model.fit(X_R,R,mu.reshape(-1))
    params = (model.coef_).reshape(X_R.shape[1],1)
    if (i+1)%10 == 0:
    print('iteration: ', i)
    print(model.coef_)

    # if np.sum(np.isnan(res_wls.params)) == 0:
    # print(res_wls.params)

    beta = params[:beta.shape[0]]
    phi = params[beta.shape[0]:p+beta.shape[0]]
    theta = params[beta.shape[0]+p:beta.shape[0]+p+q]
    print(params.shape)
    alpha = float(params[-1])

    saveDict = {'beta':beta, 'phi':phi, 'theta':theta, 'alpha':alpha}
    print(saveDict)
    return beta,phi,theta,alpha
    